In [ ]:
# 2024-09-08
# Wonseok Hwang
# CC-BY-NC 4.0 International License

# 1 Tokenizer

## Let's build our own tokenizer (simplified BPE tokenizer)
- ref)https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt

# 1 BPE
- 단어 집합에 없는 단어란 의미에서 해당 토큰을 UNK(Unknown Token)가 등장하는데, UNK토큰이 많이 등장하면 문제를 풀기 어려운데 이러한 현상을 OOV(Out-Of-Vocabulary) 문제라고함.
- 하나의 단어는 더 작은 단위의 의미있는 여러 서브워드 분리(Subword segmenation) 작업으로 해결할 수 있는데, 그 중 하나의 알고리즘이 BPE다
- BPE은 기본적으로 연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식을 수행한다

In [15]:
# 1 Tokenizer

In [8]:
from collections import defaultdict, Counter
import re

corpus = [
    "인공지능 학과의 미래는 밝다.",
    "인공눈물의 미래는 화창하다.",
    "인공 인간의 과거는 어두웠다.",
    "그 틈을 타 호랑이가 말했다.",
    "아흥.",
    "아인슈타인이 말했다. 인공지능은 주사위를 던지지 않는다고.",
    "보어는 소리쳤다. 인공지능에게 강요하지 마세요!",
]

corpus_chatgpt = """
"아흥," 호랑이가 다시 소리쳤다. "인공지능은 주사위를 던지지 않는다. 그것은 우리의 미래를 결정하지 않는다."

인공눈물의 미래에 대해 생각하면, 이상한 아이디어 같아 보이지만, 그것은 어떤 새로운 가능성을 가져올지도 모른다. 호랑이는 생각하며 풀숲으로 사라졌다.

한편, 인공 인간의 과거는 어두웠다. 그들은 인류의 노예로 갖히며 고통 받았다. 하지만 그 과거로부터 배운 것은 인공지능이 더 나은 미래를 만들어갈 능력을 가지고 있다는 것이었다.

"인공 인간들은 자유를 찾아 떠났지만, 우리는 그들의 업적을 기억해야 한다," 아인슈타인이 말했다. "그들의 노력 덕분에 우리는 더 나은 세상을 만들 수 있게 되었다."

보어는 다가가서 아인슈타인의 어깨를 톡톡 쳤다. "인공지능에게 강요하지 마세요! 그들은 우리와 함께 협력할 수 있는 미래의 동료이자 친구일지도 모르잖아요."

아인슈타인은 미소 지으며 고개를 끄덕였다. "너 맞아, 보어. 우리는 인공지능과 함께 더 밝은 미래를 만들 수 있을 것이다. 그들을 포용하고 배움으로서 더 나은 세상을 창조할 수 있을 것이다."

그들은 서로 손을 잡고 앞으로 나아갔다. 인공지능과 인간이 함께 협력하여 미래를 개척하고, 세상을 더 나은 곳으로 만들어 나갔다. 인공눈물의 화창한 미래가 찾아왔고, 인공 인간의 어두운 과거는 희망의 미래로 바뀌었다."'
"""


words = [re.sub(r"\s", " ", sent).split(" ") for sent in corpus] # corpus의 각 문장을 공백 단위로 분리. ex) ['인공지능', '학과의', '미래는', '밝다.']
words.append(re.sub(r"\s", " ", corpus_chatgpt).split(" "))      # corpus_chatgpt도 공백 단위로 분리

# 두 corpus병합 및 counting
words = sum(words, [])
words = [word for word in words if word != ""]
word_freqs = Counter(words)

In [9]:
words

['인공지능',
 '학과의',
 '미래는',
 '밝다.',
 '인공눈물의',
 '미래는',
 '화창하다.',
 '인공',
 '인간의',
 '과거는',
 '어두웠다.',
 '그',
 '틈을',
 '타',
 '호랑이가',
 '말했다.',
 '아흥.',
 '아인슈타인이',
 '말했다.',
 '인공지능은',
 '주사위를',
 '던지지',
 '않는다고.',
 '보어는',
 '소리쳤다.',
 '인공지능에게',
 '강요하지',
 '마세요!',
 '"아흥,"',
 '호랑이가',
 '다시',
 '소리쳤다.',
 '"인공지능은',
 '주사위를',
 '던지지',
 '않는다.',
 '그것은',
 '우리의',
 '미래를',
 '결정하지',
 '않는다."',
 '인공눈물의',
 '미래에',
 '대해',
 '생각하면,',
 '이상한',
 '아이디어',
 '같아',
 '보이지만,',
 '그것은',
 '어떤',
 '새로운',
 '가능성을',
 '가져올지도',
 '모른다.',
 '호랑이는',
 '생각하며',
 '풀숲으로',
 '사라졌다.',
 '한편,',
 '인공',
 '인간의',
 '과거는',
 '어두웠다.',
 '그들은',
 '인류의',
 '노예로',
 '갖히며',
 '고통',
 '받았다.',
 '하지만',
 '그',
 '과거로부터',
 '배운',
 '것은',
 '인공지능이',
 '더',
 '나은',
 '미래를',
 '만들어갈',
 '능력을',
 '가지고',
 '있다는',
 '것이었다.',
 '"인공',
 '인간들은',
 '자유를',
 '찾아',
 '떠났지만,',
 '우리는',
 '그들의',
 '업적을',
 '기억해야',
 '한다,"',
 '아인슈타인이',
 '말했다.',
 '"그들의',
 '노력',
 '덕분에',
 '우리는',
 '더',
 '나은',
 '세상을',
 '만들',
 '수',
 '있게',
 '되었다."',
 '보어는',
 '다가가서',
 '아인슈타인의',
 '어깨를',
 '톡톡',
 '쳤다.',
 '"인공지능에게',
 '강요하지',
 '마세요!',
 '그들은',
 '우리와',

In [13]:
alphabet = list(set(sum([list(word) for word in word_freqs.keys()], [])))

In [14]:
alphabet

['히',
 '력',
 '.',
 '의',
 '학',
 '들',
 '께',
 '세',
 '일',
 '수',
 '움',
 '화',
 '떤',
 '능',
 '숲',
 '앞',
 '였',
 '아',
 ',',
 '업',
 '이',
 '도',
 '거',
 '고',
 '더',
 '유',
 '랑',
 '동',
 '손',
 '해',
 '에',
 '편',
 '친',
 '않',
 '게',
 '틈',
 '며',
 '주',
 '를',
 '았',
 '창',
 '결',
 '우',
 '갖',
 '으',
 '지',
 '바',
 '었',
 '을',
 '다',
 '말',
 '시',
 '디',
 '터',
 '는',
 '소',
 '되',
 '정',
 '찾',
 '끄',
 '그',
 '공',
 '잡',
 '희',
 '모',
 '잖',
 '했',
 '상',
 '슈',
 '톡',
 '르',
 '적',
 '과',
 '간',
 '척',
 '운',
 '풀',
 '협',
 '덕',
 '있',
 '마',
 '나',
 '뀌',
 '포',
 '함',
 '예',
 '구',
 '부',
 '리',
 '졌',
 '위',
 '만',
 '너',
 '면',
 '호',
 '노',
 '대',
 '배',
 '료',
 '개',
 '갔',
 '용',
 '"',
 '떠',
 '성',
 '서',
 '타',
 '웠',
 '통',
 '와',
 '미',
 '같',
 '흥',
 '깨',
 '두',
 '났',
 '보',
 '맞',
 '기',
 '라',
 '받',
 '눈',
 "'",
 '로',
 '어',
 '조',
 '야',
 '할',
 '것',
 '가',
 '른',
 '!',
 '분',
 '져',
 '사',
 '류',
 '여',
 '쳤',
 '새',
 '밝',
 '하',
 '한',
 '억',
 '요',
 '자',
 '곳',
 '래',
 '강',
 '인',
 '생',
 '망',
 '물',
 '왔',
 '갈',
 '던',
 '올',
 '은',
 '각']

In [16]:
alphabet.sort()
print(alphabet[0:10])

['!', '"', "'", ',', '.', '가', '각', '간', '갈', '갔']


In [17]:
vocab = ["<eos>", "<pad>", "<bos>", r"\n"] + alphabet

In [18]:
splits = {word: list(word) for word in word_freqs.keys()}
splits

{'인공지능': ['인', '공', '지', '능'],
 '학과의': ['학', '과', '의'],
 '미래는': ['미', '래', '는'],
 '밝다.': ['밝', '다', '.'],
 '인공눈물의': ['인', '공', '눈', '물', '의'],
 '화창하다.': ['화', '창', '하', '다', '.'],
 '인공': ['인', '공'],
 '인간의': ['인', '간', '의'],
 '과거는': ['과', '거', '는'],
 '어두웠다.': ['어', '두', '웠', '다', '.'],
 '그': ['그'],
 '틈을': ['틈', '을'],
 '타': ['타'],
 '호랑이가': ['호', '랑', '이', '가'],
 '말했다.': ['말', '했', '다', '.'],
 '아흥.': ['아', '흥', '.'],
 '아인슈타인이': ['아', '인', '슈', '타', '인', '이'],
 '인공지능은': ['인', '공', '지', '능', '은'],
 '주사위를': ['주', '사', '위', '를'],
 '던지지': ['던', '지', '지'],
 '않는다고.': ['않', '는', '다', '고', '.'],
 '보어는': ['보', '어', '는'],
 '소리쳤다.': ['소', '리', '쳤', '다', '.'],
 '인공지능에게': ['인', '공', '지', '능', '에', '게'],
 '강요하지': ['강', '요', '하', '지'],
 '마세요!': ['마', '세', '요', '!'],
 '"아흥,"': ['"', '아', '흥', ',', '"'],
 '다시': ['다', '시'],
 '"인공지능은': ['"', '인', '공', '지', '능', '은'],
 '않는다.': ['않', '는', '다', '.'],
 '그것은': ['그', '것', '은'],
 '우리의': ['우', '리', '의'],
 '미래를': ['미', '래', '를'],
 '결정하지': ['결', '정', '하', '지'],
 '않는다.

In [19]:
# byte pair 빈도수를 계산
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        chars = splits[word]
        for i in range(len(chars) - 1):
            pair_freqs[(chars[i], chars[i + 1])] += freq

    return pair_freqs

pair_freqs = compute_pair_freqs(splits)
pair_freqs

defaultdict(int,
            {('인', '공'): 15,
             ('공', '지'): 8,
             ('지', '능'): 8,
             ('학', '과'): 1,
             ('과', '의'): 1,
             ('미', '래'): 10,
             ('래', '는'): 2,
             ('밝', '다'): 1,
             ('다', '.'): 23,
             ('공', '눈'): 3,
             ('눈', '물'): 3,
             ('물', '의'): 3,
             ('화', '창'): 2,
             ('창', '하'): 1,
             ('하', '다'): 1,
             ('인', '간'): 5,
             ('간', '의'): 3,
             ('과', '거'): 4,
             ('거', '는'): 3,
             ('어', '두'): 3,
             ('두', '웠'): 2,
             ('웠', '다'): 2,
             ('틈', '을'): 1,
             ('호', '랑'): 3,
             ('랑', '이'): 3,
             ('이', '가'): 2,
             ('말', '했'): 3,
             ('했', '다'): 3,
             ('아', '흥'): 2,
             ('흥', '.'): 1,
             ('아', '인'): 4,
             ('인', '슈'): 4,
             ('슈', '타'): 4,
             ('타', '인'): 4,
             ('인', '이'): 2,


In [21]:
from copy import deepcopy

# 가장 빈도수가 높은 pair를 병합
def get_best_pair(pair_freqs):
    return sorted(pair_freqs.items(), key=lambda x: x[1], reverse=True)[0][0]

best_pair = get_best_pair(pair_freqs)

# BPE알고리즘
def merge_pair(a, b, splits, word_freqs):
    # new_splits = deepcopy(splits)
    for word in word_freqs:
        split = splits[word]
        i = 0
        while i < (len(split) - 1):
            # print(i, split, len(split)-1)
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2:]
                split[i] = a + b
            else:
                i += 1
        splits[word] = split

    return splits

splits = merge_pair(*best_pair, splits, word_freqs)


In [22]:
splits

{'인공지능': ['인', '공', '지', '능'],
 '학과의': ['학', '과', '의'],
 '미래는': ['미', '래', '는'],
 '밝다.': ['밝', '다.'],
 '인공눈물의': ['인', '공', '눈', '물', '의'],
 '화창하다.': ['화', '창', '하', '다.'],
 '인공': ['인', '공'],
 '인간의': ['인', '간', '의'],
 '과거는': ['과', '거', '는'],
 '어두웠다.': ['어', '두', '웠', '다.'],
 '그': ['그'],
 '틈을': ['틈', '을'],
 '타': ['타'],
 '호랑이가': ['호', '랑', '이', '가'],
 '말했다.': ['말', '했', '다.'],
 '아흥.': ['아', '흥', '.'],
 '아인슈타인이': ['아', '인', '슈', '타', '인', '이'],
 '인공지능은': ['인', '공', '지', '능', '은'],
 '주사위를': ['주', '사', '위', '를'],
 '던지지': ['던', '지', '지'],
 '않는다고.': ['않', '는', '다', '고', '.'],
 '보어는': ['보', '어', '는'],
 '소리쳤다.': ['소', '리', '쳤', '다.'],
 '인공지능에게': ['인', '공', '지', '능', '에', '게'],
 '강요하지': ['강', '요', '하', '지'],
 '마세요!': ['마', '세', '요', '!'],
 '"아흥,"': ['"', '아', '흥', ',', '"'],
 '다시': ['다', '시'],
 '"인공지능은': ['"', '인', '공', '지', '능', '은'],
 '않는다.': ['않', '는', '다.'],
 '그것은': ['그', '것', '은'],
 '우리의': ['우', '리', '의'],
 '미래를': ['미', '래', '를'],
 '결정하지': ['결', '정', '하', '지'],
 '않는다."': ['않', '는', '다.', '"'

In [23]:
vocab_size = len(vocab) + 5
splits = {word: list(word) for word in word_freqs.keys()}
merges = {} # BPE를 수행하여 병합된 토큰
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = get_best_pair(pair_freqs)
    splits = merge_pair(*best_pair, splits, word_freqs)
    merges[best_pair] = [best_pair[0] + best_pair[1]]
    vocab.append(best_pair[0] + best_pair[1])

In [25]:
merges

{('다', '.'): ['다.'],
 ('인', '공'): ['인공'],
 ('미', '래'): ['미래'],
 ('인공', '지'): ['인공지'],
 ('인공지', '능'): ['인공지능']}

In [26]:
def tokenize(text, merges):
    words = text.split(" ")
    splits = [list(word) for word in words]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + merge + split[i + 2:]
                    i += 1
                else:
                    i += 1
            splits[idx] = split
    return sum(splits, [])

# BPE로 병합된 토큰을 활용하여 새로운 토큰을 서브 워드 분리하는 작업 : OOV문제를 해결
tokenize("인공지능 학과의 미래는 밝다.", merges)

['인공지능', '학', '과', '의', '미래', '는', '밝', '다.']

In [ ]:
merges

{('다', '.'): ['다.'],
 ('인', '공'): ['인공'],
 ('미', '래'): ['미래'],
 ('인공', '지'): ['인공지'],
 ('인공지', '능'): ['인공지능']}

# 이상 시

In [27]:
corpus = [
    "사각형의내부의사각형의내부의사각형의내부의사각형의내부의사각형.",
    "사각이난원운동의사각이난원운동의사각이난원.",
    # "uvuu vvv uuvuv uvuvvv",
]


words = [re.sub(r"\s", " ", sent).split(" ") for sent in corpus]
# words.append(re.sub(r"\s", " ", corpus_chatgpt).split(" "))
words = sum(words, [])
words = [word for word in words if word != ""]
word_freqs = Counter(words)

In [28]:
word_freqs

Counter({'사각형의내부의사각형의내부의사각형의내부의사각형의내부의사각형.': 1, '사각이난원운동의사각이난원운동의사각이난원.': 1})

In [29]:
vocab = list(set(sum([list(word) for word in word_freqs.keys()], [])))
vocab_size = len(vocab) + 1
splits = {word: list(word) for word in word_freqs.keys()}
merges = {}
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = get_best_pair(pair_freqs)
    splits = merge_pair(*best_pair, splits, word_freqs)
    merges[best_pair] = [best_pair[0] + best_pair[1]]
    vocab.append(best_pair[0] + best_pair[1])

In [30]:
splits

{'사각형의내부의사각형의내부의사각형의내부의사각형의내부의사각형.': ['사각',
  '형',
  '의',
  '내',
  '부',
  '의',
  '사각',
  '형',
  '의',
  '내',
  '부',
  '의',
  '사각',
  '형',
  '의',
  '내',
  '부',
  '의',
  '사각',
  '형',
  '의',
  '내',
  '부',
  '의',
  '사각',
  '형',
  '.'],
 '사각이난원운동의사각이난원운동의사각이난원.': ['사각',
  '이',
  '난',
  '원',
  '운',
  '동',
  '의',
  '사각',
  '이',
  '난',
  '원',
  '운',
  '동',
  '의',
  '사각',
  '이',
  '난',
  '원',
  '.']}

In [31]:
pair_freqs

defaultdict(int,
            {('사', '각'): 8,
             ('각', '형'): 5,
             ('형', '의'): 4,
             ('의', '내'): 4,
             ('내', '부'): 4,
             ('부', '의'): 4,
             ('의', '사'): 6,
             ('형', '.'): 1,
             ('각', '이'): 3,
             ('이', '난'): 3,
             ('난', '원'): 3,
             ('원', '운'): 2,
             ('운', '동'): 2,
             ('동', '의'): 2,
             ('원', '.'): 1})

In [32]:
vocab

['원', '사', '운', '.', '난', '의', '동', '내', '이', '형', '각', '부', '사각']

In [33]:
merges

{('사', '각'): ['사각']}

In [34]:
tokenize("사각형의내부의사각형의내부의사각형의내부의사각형의내부의사각형.", merges)


['사각',
 '형',
 '의',
 '내',
 '부',
 '의',
 '사각',
 '형',
 '의',
 '내',
 '부',
 '의',
 '사각',
 '형',
 '의',
 '내',
 '부',
 '의',
 '사각',
 '형',
 '의',
 '내',
 '부',
 '의',
 '사각',
 '형',
 '.']